In [2]:
%load_ext autoreload
%autoreload 2
import os
import matplotlib.pyplot as plt
import seaborn as sns
from os.path import join
from tqdm import tqdm
import pandas as pd
import sys
import joblib
from scipy.special import softmax
import sasc.config
import numpy as np
from collections import defaultdict
from copy import deepcopy
import pandas as pd
import story_helper
from sasc.modules.fmri_module import convert_module_num_to_voxel_num
from sasc.config import FMRI_DIR

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# load stuff
story_mapping = {
    "uts02_pilot_gpt4_mar28___ver=v4_noun___seed=1": "GenStory1_resps.npy",
    "uts02_pilot_gpt4_mar28___ver=v4_noun___seed=3": "GenStory2_resps.npy",
    "uts02_pilot_gpt4_mar28___ver=v4_noun___seed=4": "GenStory3_resps.npy",
    "uts02_pilot_gpt4_mar28___ver=v5_noun___seed=1": "GenStory4_resps.npy",
    "uts02_pilot_gpt4_mar28___ver=v5_noun___seed=2": "GenStory5_resps.npy",
    "uts02_pilot_gpt4_mar28___ver=v5_noun___seed=4": "GenStory6_resps.npy",
}

STORIES_DIR = join(sasc.config.RESULTS_DIR, "stories")
story_names = story_mapping.keys()  # os.listdir(STORIES_DIR)
cluster_neighbors = joblib.load(join(FMRI_DIR, "voxel_neighbors_and_pcs", "cluster_neighbors_v1.pkl"))

# process with timings
story_data = defaultdict(list)
for story_name in story_names:
    story_data["timing"].append(
        pd.read_csv(join(STORIES_DIR, story_name, "timings_processed.csv"))
    )
    story_data["story_name_original"].append(story_name)
    story_data["story_name_new"].append(story_mapping[story_name])
    story_data["story_text"].append(
        open(join(STORIES_DIR, story_name, "story.txt"), "r").read()
    )
    story_data["prompts"].append(
        open(join(STORIES_DIR, story_name, "prompts.txt"), "r").read().split("\n\n")
    )

    # rows
    rows = pd.read_csv(join(STORIES_DIR, story_name, "rows.csv"))
    rows["voxel_num"] = rows.apply(
        lambda row: convert_module_num_to_voxel_num(row["module_num"], row["subject"]),
        axis=1,
    )
    rows = rows[
        [
            "expl",
            "module_num",
            "top_explanation_init_strs",
            "subject",
            "fmri_test_corr",
            "top_score_synthetic",
            "roi_anat",
            "roi_func",
            "voxel_num",
        ]
    ]
    rows['cluster_nums'] = rows['voxel_num'].map(cluster_neighbors)
    story_data["rows"].append(rows)


joblib.dump(story_data, "../results/pilot_story_data.pkl")

['../results/pilot_story_data.pkl']

In [5]:
loo_pc = joblib.load(f'{FMRI_DIR}/voxel_neighbors_and_pcs/loo_pc_UTS02.pkl')